In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.experiment.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        - '06_12_24_WMH_FullMultiAnnoVol'

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: False 

<IPython.core.display.Javascript object>

In [3]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=True,
)

Finished loading inference stats.
Log amounts: root                                                                                             log_set                                              
/storage/vbutoi/scratch/ESE/inference/06_12_24_WMH_FullMultiAnnoVol/WMH_Individual_Uncalibrated  20240612_085239-WVBT-9dccca4ef8afec21bca192030f75d2f1    19795
                                                                                                 20240612_085243-02NK-1334ebc13752b362cdeef2fdd4b66604    19795
                                                                                                 20240612_085248-V8CE-b3a5b8910b19a1589e7e2ac95c5a1681    19795
                                                                                                 20240612_085252-LF35-09c30a291004abade2e995305a5e1c0e    19795
                                                                                                 20240612_085255-A64D-ee4f8a434727eec178bed49efca3de53    13282
  

In [4]:
# Next let's make our predictions into their corresponding volumes.
slice_df = inference_df.groupby([
    "annotator",
    "data_id",
    "pretrained_seed", 
    "slice_idx",
    "split",
    "task"
])[["gt_volume", "soft_volume", "hard_volume"]].mean().reset_index()

In [5]:
slice_df

,annotator,data_id,pretrained_seed,slice_idx,split,task,gt_volume,soft_volume,hard_volume
0,observer_o12,101,40,0,cal,Amsterdam,4.0,8.332066,6.0
1,observer_o12,101,40,1,cal,Amsterdam,4.0,4.088889,1.0
2,observer_o12,101,40,2,cal,Amsterdam,4.0,1.993716,0.0
3,observer_o12,101,40,3,cal,Amsterdam,2.0,4.903453,3.0
4,observer_o12,101,40,4,cal,Amsterdam,2.0,4.235747,2.0
...,...,...,...,...,...,...,...,...,...
16803,observer_o4,69,43,58,val,Singapore,139.0,268.495728,249.0
16804,observer_o4,69,43,59,val,Singapore,139.0,226.608490,209.0
16805,observer_o4,69,43,60,val,Singapore,176.0,200.179092,176.0
16806,observer_o4,69,43,61,val,Singapore,176.0,202.882385,177.0


In [6]:
slice_df.select(data_id='101', slice_idx=0, pretrained_seed=40)

,annotator,data_id,pretrained_seed,slice_idx,split,task,gt_volume,soft_volume,hard_volume
0,observer_o12,101,40,0,cal,Amsterdam,4.0,8.332066,6.0
10216,observer_o3,101,40,0,cal,Amsterdam,4.0,7.763233,2.0
13620,observer_o4,101,40,0,cal,Amsterdam,4.0,7.763233,2.0


In [7]:
# Make two new columns, one for the soft volume error and one for the hard volume error.
volume_df['soft_volume_error'] = volume_df['soft_volume'] - volume_df['gt_volume']
volume_df['hard_volume_error'] = volume_df['hard_volume'] - volume_df['gt_volume']

NameError: name 'volume_df' is not defined

In [ ]:
melted_volume_df = pd.melt(
    volume_df,
    id_vars=["annotator", "data_id", "pretrained_seed", "task"], 
    value_vars=["gt_volume", "soft_volume", "hard_volume"],
    var_name="volume_type",
    value_name="volume",
)

In [ ]:
melted_error_df = pd.melt(
    volume_df,
    id_vars=["annotator", "data_id", "pretrained_seed", "task", "gt_volume", "soft_volume", "hard_volume"],
    value_vars=["soft_volume_error", "hard_volume_error"],
    var_name="volume_type",
    value_name="error",
)

In [ ]:
import numpy as np

# Make some columns that are useful for plotting.
melted_error_df['abs_error'] = melted_error_df['error'].abs()
melted_error_df['log_abs_error'] = melted_error_df['error'].abs().apply(lambda x: np.log(x + 1))

# Experiment 1: Looking at one annotator on Amsterdam, let's look at how the volumetric comparison looks like.

In [ ]:
exp_1_df = melted_error_df.select(annotator='observer_o12', task='Amsterdam')

In [ ]:
g = sns.catplot(
    exp_1_df,
    x="data_id",
    y="error",
    hue="volume_type",
    aspect=3,
    height=8,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Soft/Hard Volumetric Error', fontsize=30)

# Show the plot
plt.show()

In [ ]:
g = sns.catplot(
    melted_volume_df.select(data_id='101'),
    x="volume_type",
    y="volume",
    hue="volume_type",
    height=8,
    sharey=False,
    kind="box"
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Soft/Hard Volume', fontsize=30)

# Show the plot
plt.show()

In [ ]:
g = sns.catplot(
    exp_1_df,
    x="data_id",
    y="log_abs_error",
    hue="volume_type",
    aspect=3,
    height=8,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Absolute Soft/Hard Volumetric Log Error', fontsize=30)

# Show the plot
plt.show()

# Experiment 2: Let's look at the same thing but this time also for Singapore.

In [ ]:
exp_2_df = melted_error_df.select(annotator='observer_o12', task='Singapore')

In [ ]:
g = sns.catplot(
    exp_2_df,
    x="data_id",
    y="error",
    hue="volume_type",
    aspect=3,
    height=8,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Singapore Soft/Hard Volumetric Error', fontsize=30)

# Show the plot
plt.show()

In [ ]:
g = sns.catplot(
    exp_2_df,
    x="data_id",
    y="log_abs_error",
    hue="volume_type",
    aspect=3,
    height=8,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Singapore Absolute Soft/Hard Volumetric Log Error', fontsize=30)

# Show the plot
plt.show()

# Experiment 3: The interesting thing about WMH is that we have multiple annotations per data_id (for some of the data_ids). Let's gather all of the data_ids that have all three annotators.

In [ ]:
# Get the subset of the dataframe for which the number of unique annotators for each data_id is 3

# Step 1: Group by 'data_id' and count unique 'annotator' values
unique_counts = melted_error_df.groupby('data_id')['annotator'].nunique()
# Step 2: Filter 'data_id's that have exactly three unique 'annotator' values
filtered_data_ids = unique_counts[unique_counts == 3].index
# Step 3: Get the subset of rows with the filtered 'data_id's
multianno_melted_error_df= melted_error_df[melted_error_df['data_id'].isin(filtered_data_ids)]

In [ ]:
multianno_melted_error_df['data_id'].unique().shape[0]

In [ ]:
multianno_melted_error_df.select(data_id='101', pretrained_seed=40)